In [1]:
import tensorflow as tf

# Wichtige Hyperparameter (können von Nutzer:innen angepasst werden)
EPOCHS = 50          # Anzahl der Trainings-Epochen (empfohlen ~50 für gute Ergebnisse)
BATCH_SIZE = 256     # Batch-Größe
NOISE_DIM = 100      # Dimension des Eingaberauschens für den Generator
LEARNING_RATE = 0.0002   # Lernrate für Adam Optimizer
BETA_1 = 0.5            # Beta1 für Adam (momentum term)

# MNIST Datensatz laden und vorbereiten
(train_images, _), _ = tf.keras.datasets.mnist.load_data()
train_images = train_images.reshape(-1, 28, 28, 1).astype("float32")
# Wertebereich [0,255] -> [-1,1]
train_images = (train_images - 127.5) / 127.5
train_dataset = tf.data.Dataset.from_tensor_slices(train_images).shuffle(len(train_images)).batch(BATCH_SIZE)

# Generator-Architektur (vollständig vernetzt)
generator_dense_mnist = tf.keras.Sequential([
    tf.keras.layers.Dense(256, use_bias=False, input_shape=(NOISE_DIM,)),  # Dense-Schicht 1
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.LeakyReLU(alpha=0.2),
    tf.keras.layers.Dense(512, use_bias=False),                            # Dense-Schicht 2 (kann angepasst oder weitere hinzugefügt werden)
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.LeakyReLU(alpha=0.2),
    tf.keras.layers.Dense(28*28*1, activation='tanh'),                     # Ausgabeschicht: 784 Neuronen für 28x28 Bild, Tanh-Aktivierung
    tf.keras.layers.Reshape((28, 28, 1))
], name="generator_dense_mnist")

# Diskriminator-Architektur (vollständig vernetzt)
discriminator_dense_mnist = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28, 1)),   # Bild in Vektor umwandeln
    tf.keras.layers.Dense(512),
    tf.keras.layers.LeakyReLU(alpha=0.2),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(256),
    tf.keras.layers.LeakyReLU(alpha=0.2),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(1)  # Ausgabe: ein Neuron (Echtheits-Wahrscheinlichkeit als Logit)
], name="discriminator_dense_mnist")

# Optimizer und Loss-Funktionen (Binary Crossentropy als Verlust, mit Logits)
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)
# Diskriminator-Loss: minimiert real_vs_fake-Fehlklassifikationen
def discriminator_loss(real_output, fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)   # echte Bilder -> Label 1
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)  # generierte Bilder -> Label 0
    return real_loss + fake_loss

# Generator-Loss: Generator will Diskriminator täuschen (Diskriminator soll "echt" sagen)
def generator_loss(fake_output):
    return cross_entropy(tf.ones_like(fake_output), fake_output)

# Optimizer für beide Netzwerke (Adam wie empfohlen)
generator_optimizer = tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE, beta_1=BETA_1)
discriminator_optimizer = tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE, beta_1=BETA_1)


2025-06-04 14:34:17.560856: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-06-04 14:34:17.581177: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-06-04 14:34:17.587400: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-06-04 14:34:17.602804: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-06-04 14:34:18.692968: W tensorflow/compiler/tf2

In [2]:
import numpy as np

# Trainings-Loop für Dense-GAN auf MNIST
EPOCHS_DEMO = 10  # (Für Demo-Zwecke hier 10 Epochen; für bessere Ergebnisse auf 50 erhöhen)
fixed_seed = tf.random.normal([16, NOISE_DIM])  # Fester Seed, um Ergebnisse über Epochen vergleichbar zu machen

for epoch in range(EPOCHS_DEMO):
    for real_images in train_dataset:
        noise = tf.random.normal([real_images.shape[0], NOISE_DIM])
        # Schritt 1: Generator erzeugt Fake-Bilder
        with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
            generated_images = generator_dense_mnist(noise, training=True)
            # Diskriminator bewertet echte und gefälschte Bilder
            real_output = discriminator_dense_mnist(real_images, training=True)
            fake_output = discriminator_dense_mnist(generated_images, training=True)
            # Verluste berechnen
            gen_loss = generator_loss(fake_output)
            disc_loss = discriminator_loss(real_output, fake_output)
        # Schritt 2: Backpropagation und Gewichts-Updates
        gradients_of_generator = gen_tape.gradient(gen_loss, generator_dense_mnist.trainable_variables)
        gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator_dense_mnist.trainable_variables)
        generator_optimizer.apply_gradients(zip(gradients_of_generator, generator_dense_mnist.trainable_variables))
        discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator_dense_mnist.trainable_variables))
    # Am Ende jeder Epoche: Verlustwerte ausgeben
    print(f"Epoche {epoch+1}/{EPOCHS_DEMO} - Generator-Loss: {gen_loss:.4f}, Diskriminator-Loss: {disc_loss:.4f}")
    # Optional: Beispielbilder generieren (alle 5 Epochen)
    if (epoch + 1) % 5 == 0:
        preds = generator_dense_mnist(fixed_seed, training=False).numpy()
        preds = (preds * 0.5) + 0.5  # Reskalierung von [-1,1] zurück auf [0,1]
        # Ausgabe von einer generierten Ziffer als grobe visuelle Kontrolle (als Zahlenmatrix)
        sample_img = preds[0].reshape(28, 28)
        print("Beispiel generiertes Bild (Wertematrix):")
        print(np.round(sample_img, 2))


2025-05-19 15:21:04.437706: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Epoche 1/10 - Generator-Loss: 1.0451, Diskriminator-Loss: 1.1812


2025-05-19 15:21:25.055819: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Epoche 2/10 - Generator-Loss: 1.0088, Diskriminator-Loss: 1.0023
Epoche 3/10 - Generator-Loss: 1.7314, Diskriminator-Loss: 1.3412


2025-05-19 15:22:06.210383: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Epoche 4/10 - Generator-Loss: 1.0398, Diskriminator-Loss: 1.0182
Epoche 5/10 - Generator-Loss: 0.9469, Diskriminator-Loss: 0.9229
Beispiel generiertes Bild (Wertematrix):
[[0.31 0.   0.03 0.   0.01 0.03 0.02 0.02 0.24 0.01 0.03 0.02 0.   0.01
  0.03 0.15 0.85 0.05 0.01 0.   0.07 0.05 0.03 0.04 0.02 0.03 0.04 0.06]
 [0.01 0.01 0.01 0.03 0.01 0.01 0.01 0.   0.08 0.01 0.   0.01 0.03 0.
  0.09 0.   0.   0.01 0.01 0.01 0.32 0.   0.01 0.03 0.   0.1  0.13 0.03]
 [0.01 0.   0.01 0.08 0.01 0.01 0.02 0.01 0.   0.   0.06 0.01 0.12 0.
  0.01 0.01 0.1  0.02 0.   0.03 0.02 0.01 0.02 0.01 0.07 0.11 0.07 0.  ]
 [0.   0.09 0.   0.01 0.11 0.   0.01 0.08 0.   0.03 0.   0.   0.   0.
  0.   0.   0.   0.   0.   0.   0.03 0.12 0.1  0.   0.01 0.02 0.01 0.  ]
 [0.07 0.   0.   0.02 0.01 0.03 0.04 0.04 0.01 0.   0.01 0.   0.   0.
  0.   0.   0.   0.   0.   0.   0.01 0.01 0.11 0.03 0.   0.01 0.36 0.02]
 [0.02 0.05 0.   0.16 0.17 0.01 0.04 0.04 0.03 0.04 0.01 0.05 0.   0.
  0.   0.   0.02 0.   0.   0.   0.02 0.   

2025-05-19 15:23:28.551091: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Epoche 8/10 - Generator-Loss: 1.0056, Diskriminator-Loss: 1.0584
Epoche 9/10 - Generator-Loss: 1.1314, Diskriminator-Loss: 1.0255
Epoche 10/10 - Generator-Loss: 1.4397, Diskriminator-Loss: 0.9663
Beispiel generiertes Bild (Wertematrix):
[[0.01 0.   0.   0.   0.   0.34 0.   0.   0.   0.03 0.   0.   0.   0.01
  0.02 0.   0.01 0.   0.01 0.   0.04 0.05 0.02 0.   0.05 0.   0.02 0.03]
 [0.01 0.   0.   0.   0.   0.01 0.   0.   0.01 0.   0.   0.   0.   0.03
  0.   0.   0.   0.23 0.01 0.01 0.04 0.01 0.   0.03 0.   0.05 0.09 0.  ]
 [0.05 0.01 0.   0.01 0.17 0.01 0.04 0.   0.01 0.02 0.   0.01 0.   0.01
  0.02 0.01 0.01 0.   0.01 0.   0.   0.   0.06 0.03 0.16 0.   0.01 0.04]
 [0.   0.   0.01 0.58 0.   0.   0.84 0.01 0.02 0.05 0.03 0.41 0.03 0.03
  0.03 0.   0.03 0.   0.   0.01 0.   0.14 0.04 0.13 0.   0.   0.01 0.  ]
 [0.   0.   0.   0.   0.   0.   0.02 0.04 0.   0.01 0.05 0.08 0.05 0.07
  0.01 0.05 0.02 0.01 0.06 0.15 0.01 0.03 0.   0.04 0.02 0.02 0.   0.  ]
 [0.   0.   0.   0.   0.02 0.02 0.01 0

In [3]:
# Generator-Architektur (DCGAN für MNIST)
generator_dcgan_mnist = tf.keras.Sequential([
    tf.keras.layers.Dense(7*7*256, use_bias=False, input_shape=(NOISE_DIM,)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.LeakyReLU(alpha=0.2),
    tf.keras.layers.Reshape((7, 7, 256)),  # Startform: 7x7x256

    tf.keras.layers.Conv2DTranspose(128, (5,5), strides=(1,1), padding='same', use_bias=False),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.LeakyReLU(alpha=0.2),
    # Jetzt 7x7x128

    tf.keras.layers.Conv2DTranspose(64, (5,5), strides=(2,2), padding='same', use_bias=False),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.LeakyReLU(alpha=0.2),
    # Jetzt 14x14x64

    tf.keras.layers.Conv2DTranspose(1, (5,5), strides=(2,2), padding='same', use_bias=False, activation='tanh')
    # Ausgabe: 28x28x1
], name="generator_dcgan_mnist")

# Diskriminator-Architektur (DCGAN für MNIST)
discriminator_dcgan_mnist = tf.keras.Sequential([
    tf.keras.layers.Conv2D(64, (5,5), strides=(2,2), padding='same', input_shape=(28,28,1)),
    tf.keras.layers.LeakyReLU(alpha=0.2),
    tf.keras.layers.Dropout(0.3),

    tf.keras.layers.Conv2D(128, (5,5), strides=(2,2), padding='same'),
    tf.keras.layers.LeakyReLU(alpha=0.2),
    tf.keras.layers.Dropout(0.3),

    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(1)
], name="discriminator_dcgan_mnist")

# Loss-Funktionen und Optimizer können wiederverwendet werden (bereits definiert als generator_loss, discriminator_loss, etc.)
generator_optimizer_dc = tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE, beta_1=BETA_1)
discriminator_optimizer_dc = tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE, beta_1=BETA_1)


/opt/conda/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [4]:
# Trainings-Loop für DCGAN auf MNIST
EPOCHS_DEMO = 10  # kurze Demo (für sinnvolle Ergebnisse könnte man 30-50 Epochen nutzen)
fixed_seed = tf.random.normal([16, NOISE_DIM])

for epoch in range(EPOCHS_DEMO):
    for real_images in train_dataset:
        noise = tf.random.normal([real_images.shape[0], NOISE_DIM])
        with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
            fake_images = generator_dcgan_mnist(noise, training=True)
            real_output = discriminator_dcgan_mnist(real_images, training=True)
            fake_output = discriminator_dcgan_mnist(fake_images, training=True)
            gen_loss = generator_loss(fake_output)
            disc_loss = discriminator_loss(real_output, fake_output)
        gradients_gen = gen_tape.gradient(gen_loss, generator_dcgan_mnist.trainable_variables)
        gradients_disc = disc_tape.gradient(disc_loss, discriminator_dcgan_mnist.trainable_variables)
        generator_optimizer_dc.apply_gradients(zip(gradients_gen, generator_dcgan_mnist.trainable_variables))
        discriminator_optimizer_dc.apply_gradients(zip(gradients_disc, discriminator_dcgan_mnist.trainable_variables))
    print(f"Epoche {epoch+1}/{EPOCHS_DEMO} - Gen.Loss: {gen_loss:.4f}, Disc.Loss: {disc_loss:.4f}")


2025-05-19 15:24:19.640389: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:531] Loaded cuDNN version 8907
W0000 00:00:1747668259.848471    3479 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1747668259.908202    3479 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1747668259.921127    3479 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1747668259.921919    3479 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1747668259.922811    3479 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1747668259.925737    3479 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1747668259.926522    3479 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1747668259.927410    3479 gpu_t

Epoche 1/10 - Gen.Loss: 0.7080, Disc.Loss: 1.3484
Epoche 2/10 - Gen.Loss: 0.9697, Disc.Loss: 1.4559
Epoche 3/10 - Gen.Loss: 0.7288, Disc.Loss: 1.3190
Epoche 4/10 - Gen.Loss: 0.6918, Disc.Loss: 1.3700
Epoche 5/10 - Gen.Loss: 0.7847, Disc.Loss: 1.2164


2025-05-19 15:27:59.281027: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Epoche 6/10 - Gen.Loss: 0.7323, Disc.Loss: 1.3256
Epoche 7/10 - Gen.Loss: 0.7735, Disc.Loss: 1.2844
Epoche 8/10 - Gen.Loss: 0.7104, Disc.Loss: 1.2554
Epoche 9/10 - Gen.Loss: 0.7354, Disc.Loss: 1.2871
Epoche 10/10 - Gen.Loss: 0.6658, Disc.Loss: 1.3041


In [5]:
# Fashion-MNIST Daten laden
(train_images_f, _), _ = tf.keras.datasets.fashion_mnist.load_data()
train_images_f = train_images_f.reshape(-1, 28, 28, 1).astype("float32")
train_images_f = (train_images_f - 127.5) / 127.5
train_dataset_f = tf.data.Dataset.from_tensor_slices(train_images_f).shuffle(len(train_images_f)).batch(BATCH_SIZE)

# Generator und Diskriminator für Fashion-MNIST (identisch zum MNIST Dense-GAN)
generator_dense_fashion = tf.keras.models.clone_model(generator_dense_mnist)  # Klonen wir das obige Modell
discriminator_dense_fashion = tf.keras.models.clone_model(discriminator_dense_mnist)
# (Alternativ könnte man sie auch neu definieren; hier kopieren wir die Struktur für Konsistenz)

# Optimizer auch hier anlegen
generator_optimizer_f = tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE, beta_1=BETA_1)
discriminator_optimizer_f = tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE, beta_1=BETA_1)


29515/29515 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
26421880/26421880 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step
5148/5148 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
4422102/4422102 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [6]:
EPOCHS_DEMO = 10  # Demo-Trainingsepochen (für bessere Ergebnisse erhöhen)
fixed_seed = tf.random.normal([16, NOISE_DIM])

for epoch in range(EPOCHS_DEMO):
    for real_images in train_dataset_f:
        noise = tf.random.normal([real_images.shape[0], NOISE_DIM])
        with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
            fake_images = generator_dense_fashion(noise, training=True)
            real_output = discriminator_dense_fashion(real_images, training=True)
            fake_output = discriminator_dense_fashion(fake_images, training=True)
            gen_loss = generator_loss(fake_output)
            disc_loss = discriminator_loss(real_output, fake_output)
        grads_gen = gen_tape.gradient(gen_loss, generator_dense_fashion.trainable_variables)
        grads_disc = disc_tape.gradient(disc_loss, discriminator_dense_fashion.trainable_variables)
        generator_optimizer_f.apply_gradients(zip(grads_gen, generator_dense_fashion.trainable_variables))
        discriminator_optimizer_f.apply_gradients(zip(grads_disc, discriminator_dense_fashion.trainable_variables))
    print(f"Epoche {epoch+1}/{EPOCHS_DEMO} - Gen.Loss: {gen_loss:.4f}, Disc.Loss: {disc_loss:.4f}")


Epoche 1/10 - Gen.Loss: 2.1752, Disc.Loss: 0.7705
Epoche 2/10 - Gen.Loss: 1.2227, Disc.Loss: 0.8587
Epoche 3/10 - Gen.Loss: 1.2990, Disc.Loss: 0.9085
Epoche 4/10 - Gen.Loss: 1.0102, Disc.Loss: 1.1376
Epoche 5/10 - Gen.Loss: 1.4480, Disc.Loss: 1.0291
Epoche 6/10 - Gen.Loss: 1.3882, Disc.Loss: 0.7502
Epoche 7/10 - Gen.Loss: 1.1175, Disc.Loss: 0.9366
Epoche 8/10 - Gen.Loss: 1.0757, Disc.Loss: 1.0676
Epoche 9/10 - Gen.Loss: 1.4679, Disc.Loss: 0.9901
Epoche 10/10 - Gen.Loss: 1.0355, Disc.Loss: 1.1435


In [7]:
# Generator (DCGAN) für Fashion-MNIST
generator_dcgan_fashion = tf.keras.Sequential([
    tf.keras.layers.Dense(7*7*256, use_bias=False, input_shape=(NOISE_DIM,)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.LeakyReLU(alpha=0.2),
    tf.keras.layers.Reshape((7, 7, 256)),

    tf.keras.layers.Conv2DTranspose(128, (5,5), strides=(1,1), padding='same', use_bias=False),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.LeakyReLU(alpha=0.2),

    tf.keras.layers.Conv2DTranspose(64, (5,5), strides=(2,2), padding='same', use_bias=False),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.LeakyReLU(alpha=0.2),

    tf.keras.layers.Conv2DTranspose(1, (5,5), strides=(2,2), padding='same', use_bias=False, activation='tanh')
], name="generator_dcgan_fashion")

# Diskriminator (DCGAN) für Fashion-MNIST
discriminator_dcgan_fashion = tf.keras.Sequential([
    tf.keras.layers.Conv2D(64, (5,5), strides=(2,2), padding='same', input_shape=(28,28,1)),
    tf.keras.layers.LeakyReLU(alpha=0.2),
    tf.keras.layers.Dropout(0.3),

    tf.keras.layers.Conv2D(128, (5,5), strides=(2,2), padding='same'),
    tf.keras.layers.LeakyReLU(alpha=0.2),
    tf.keras.layers.Dropout(0.3),

    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(1)
], name="discriminator_dcgan_fashion")

# Optimizer initialisieren
generator_optimizer_fdc = tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE, beta_1=BETA_1)
discriminator_optimizer_fdc = tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE, beta_1=BETA_1)


In [8]:
EPOCHS_DEMO = 10  # Demo-Epochen, bei Bedarf erhöhen
fixed_seed = tf.random.normal([16, NOISE_DIM])

for epoch in range(EPOCHS_DEMO):
    for real_images in train_dataset_f:
        noise = tf.random.normal([real_images.shape[0], NOISE_DIM])
        with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
            fake_images = generator_dcgan_fashion(noise, training=True)
            real_output = discriminator_dcgan_fashion(real_images, training=True)
            fake_output = discriminator_dcgan_fashion(fake_images, training=True)
            gen_loss = generator_loss(fake_output)
            disc_loss = discriminator_loss(real_output, fake_output)
        grads_gen = gen_tape.gradient(gen_loss, generator_dcgan_fashion.trainable_variables)
        grads_disc = disc_tape.gradient(disc_loss, discriminator_dcgan_fashion.trainable_variables)
        generator_optimizer_fdc.apply_gradients(zip(grads_gen, generator_dcgan_fashion.trainable_variables))
        discriminator_optimizer_fdc.apply_gradients(zip(grads_disc, discriminator_dcgan_fashion.trainable_variables))
    print(f"Epoche {epoch+1}/{EPOCHS_DEMO} - Gen.Loss: {gen_loss:.4f}, Disc.Loss: {disc_loss:.4f}")


Epoche 1/10 - Gen.Loss: 0.7292, Disc.Loss: 1.3198


2025-05-19 15:34:49.531467: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Epoche 2/10 - Gen.Loss: 0.7590, Disc.Loss: 1.2315
Epoche 3/10 - Gen.Loss: 0.7510, Disc.Loss: 1.3079
Epoche 4/10 - Gen.Loss: 0.8819, Disc.Loss: 1.1129
Epoche 5/10 - Gen.Loss: 1.2157, Disc.Loss: 1.0982
Epoche 6/10 - Gen.Loss: 1.0652, Disc.Loss: 1.0788
Epoche 7/10 - Gen.Loss: 0.8183, Disc.Loss: 1.3111
Epoche 8/10 - Gen.Loss: 0.7347, Disc.Loss: 1.2947
Epoche 9/10 - Gen.Loss: 0.7634, Disc.Loss: 1.1932
Epoche 10/10 - Gen.Loss: 0.7897, Disc.Loss: 1.3673
